In [52]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from patsy import dmatrix
from sklearn.metrics import mean_squared_error
from pyearth import Earth
from numpy import mean
from numpy import absolute
from numpy import sqrt

# Q1 {-}
Find the number of degrees of freedom of the followng models. Exclude the intercept when counting the degrees of freedom. You may either show your calculation, or explain briefly how you are computing the degrees of freedom.

(a) A model with one predictor, where the predictor is transformed into a quadratic spline with 5 knots \
(b) A model with one predictor, where the predictor is transformed into a natural cubic spline with 4 knots \
(c) A model with four predictors, where the transformations of the respective predictors are (i) cubic spline transformation with 3 knots, (ii) log transformation, (iii) linear spline transformation with 2 knots, (iv) polynomial transformation of degree 4.

*(2 + 2 + 6 points)*

Computing the degrees of freedom:
    
    1. df = k + 2 = 5 +2 = 7
   
   2. df = k + 1 = 4 + 1 = 5
   
   3. 
        i. df = k + 3 = 6
        
        ii. df = 1 (log transformation)
       
       iii. df = k + 1 = 3
       
       iv. df = 4 (polynomial transformation with no knots)

model with four predictors:
        df = 6 + 1 + 3 + 4 = 14

# Q2 {-}
Find the number of knots in the following spline transformations, if each transformation corresponds to 7 degrees of freedom (excluding the intercept).

(a) Cubic spline transformation \
(b) Natural cubic spline transformation \
(c) Spline transformation of degree 4

*(3 points)*

a. cubic spline transformation has 4 knots

b. natural cubic spline transformation has 2 knots

c. spline transformaition of degree 4 has 3 knots

# Q3 {-}
Read the datasets *house_feature_train.csv, house_price_train.csv, house_feature_test.csv* and *house_price_test.csv*. Develop a generalized additive model (GAM) to predict *house_price* based on all the predictors except *house_id*. Use the following transformations:

(a) Log transformation of *house_price* \
(b) Natural cubic spline transformation of *house_age* with 4 degrees of freedom \
(c) Log transformation of *distance_MRT* \
(d) Natural cubic spline transformation of *number_convenience_stores* with 4 degrees of freedom \

Use no transformations for *latitude* and *longitude*. Use the datasets with the *'_train'* suffix to develop the model.

What is the adjusted R-squared of the developed model?

*(4 points for code, 1 point for answer)*

In [68]:
price_train = pd.read_csv("house_price_train.csv")
price_test = pd.read_csv("house_price_test.csv")
feature_train = pd.read_csv("house_feature_train.csv")
feature_test = pd.read_csv("house_feature_test.csv")

train = pd.merge(price_train, feature_train)
test = pd.merge(price_test, feature_test)

In [20]:
y = np.log(train["house_price"])

transformed_x = dmatrix("cr(house_age, df = 4, constraints = 'center') + np.log(distance_MRT) + cr(number_convenience_stores, df = 4, constraints = 'center') + longitude + latitude",
                        data = {'house_age':train['house_age'], 'distance_MRT':train['distance_MRT'], 'number_convenience_stores':train['number_convenience_stores'], 'longitude':train['longitude'], 'latitude':train['latitude']},return_type = 'dataframe')

model = sm.OLS(y, transformed_x).fit()

In [21]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            house_price   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     115.5
Date:                Wed, 13 Apr 2022   Prob (F-statistic):           6.59e-94
Time:                        22:33:35   Log-Likelihood:                -79.132
No. Observations:                 275   AIC:                             182.3
Df Residuals:                     263   BIC:                             225.7
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
================================================================================================================================
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------
Intercept                                                    -1102.5466    211.858     -5.204      0.000   -1519.700    -685.393
cr(house_age, df=4, constraints='center')[0]                    -0.1271      0.061     -2.093      0.037      -0.247      -0.008
cr(house_age, df=4, constraints='center')[1]                    -0.2812      0.049     -5.758      0.000      -0.377      -0.185
cr(house_age, df=4, constraints='center')[2]                    -0.3542      0.059     -5.988      0.000      -0.471      -0.238
cr(house_age, df=4, constraints='center')[3]                     0.1912      0.125      1.529      0.128      -0.055       0.438
np.log(distance_MRT)                                            -0.3048      0.032     -9.534      0.000      -0.368      -0.242
cr(number_convenience_stores, df=4, constraints='center')[0]    -0.0034      0.053     -0.064      0.949      -0.108       0.101
cr(number_convenience_stores, df=4, constraints='center')[1]     0.1901      0.047      4.073      0.000       0.098       0.282
cr(number_convenience_stores, df=4, constraints='center')[2]     0.1460      0.061      2.404      0.017       0.026       0.266
cr(number_convenience_stores, df=4, constraints='center')[3]     0.1014      0.109      0.929      0.354      -0.114       0.316
longitude                                                        4.5247      1.769      2.557      0.011       1.041       8.008
latitude                                                        22.4979      2.027     11.098      0.000      18.506      26.489
==============================================================================
Omnibus:                        2.942   Durbin-Watson:                   2.131
Prob(Omnibus):                  0.230   Jarque-Bera (JB):                2.635
Skew:                          -0.185   Prob(JB):                        0.268
Kurtosis:                       3.305   Cond. No.                     1.32e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.32e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**The adjusted r-squared for this developed model is 0.821.**

# Q4 {-}
Find the RMSE of the model developed in the previous question on the test data (datasets with the *'_test'* suffix). Assume all variables in the *train* and *test* datasets have the same distribution.

*(3 points for code, 1 point for answer)*

In [42]:
test_x = dmatrix("cr(house_age, df = 4, constraints = 'center') + np.log(distance_MRT) + cr(number_convenience_stores, df = 4, constraints = 'center') + longitude + latitude",
                        data = {'house_age':test['house_age'], 'distance_MRT':test['distance_MRT'], 'number_convenience_stores':test['number_convenience_stores'], 'longitude':test['longitude'], 'latitude':test['latitude']},return_type = 'dataframe')

In [44]:
np.sqrt(mean_squared_error(np.exp(model.predict(test_x)), test.house_price))

367.1329627411743

**The RMSE of the model developed is 367.13.**

# Q5 {-}
Use 5-fold cross validation to optimize the degrees of freedom of the natural cubic spline transformations of *house_age* and *number_convenience_stores* in the model developed in Q3. Report the optimal degrees of freedom corresponding to both the predictors.

Use the *sklearn* functions for 5-fold cross validation. Here is a good example of *K*-fold cross validation using *sklearn*: https://www.statology.org/k-fold-cross-validation-in-python/ 

**Hint**: Consider degrees of freedom ranging from 2 to 10 for the spline transformation of each of the two predictors -  *house_age* and *number_convenience_stores*

*(7 points for code, 1 point for answer)*

In [50]:
y = np.log(train['house_price'])
cv_score = {}
for i in range(2,11):
    for j in range(2, 11):
        tranformed_x = dmatrix(f"cr(house_age, df = {i}, constraints = 'center')+np.log(distance_MRT)+cr(number_convenience_stores, df = {j}, constraints = 'center')+latitude+longitude",
                               data = {'house_age': train['house_age'], 'distance_MRT': train['distance_MRT'], 'number_convenience_stores': train['number_convenience_stores'], 'latitude': train['latitude'], 'longitude': train['longitude']},
                               return_type = 'dataframe')
        cv = KFold(n_splits=5, random_state=1, shuffle=True) 
        model = LinearRegression()
        scores = cross_val_score(model, transformed_x, y, scoring='neg_mean_squared_error',
                                 cv=cv, n_jobs=-1)
        cv_score[i,j] = np.mean(-scores)

In [54]:
combo = pd.Series(cv_score)
combo.sort_values(ascending = True)

2   2     0.116707
8   5     0.116707
    4     0.116707
    3     0.116707
    2     0.116707
            ...   
4   7     0.116707
    6     0.116707
    5     0.116707
6   5     0.116707
10  10    0.116707
Length: 81, dtype: float64

**As you can see above, the most optimal df's are 2 degrees of freedom for both house_age and number_convenience_stores.**

# Q6 {-}
Find the RMSE on test data, based on the optimized model developed in Q6. Assume all variables in the *train* and *test* datasets have the same distribution.

*(4 points for code, 1 point for answer)*

In [56]:
transformed_x = dmatrix("cr(house_age, df = 2, constraints = 'center') + np.log(distance_MRT) + cr(number_convenience_stores, df = 2, constraints = 'center') + longitude + latitude",
                        data = {'house_age':train['house_age'], 'distance_MRT':train['distance_MRT'], 'number_convenience_stores':train['number_convenience_stores'], 'longitude':train['longitude'], 'latitude':train['latitude']},return_type = 'dataframe')

model = sm.OLS(y, transformed_x).fit()

test_x = dmatrix("cr(house_age, df = 2, constraints = 'center') + np.log(distance_MRT) + cr(number_convenience_stores, df = 2, constraints = 'center') + longitude + latitude",
                        data = {'house_age':test['house_age'], 'distance_MRT':test['distance_MRT'], 'number_convenience_stores':test['number_convenience_stores'], 'longitude':test['longitude'], 'latitude':test['latitude']},return_type = 'dataframe')

np.sqrt(mean_squared_error(np.exp(model.predict(test_x)), test.house_price))

353.71403196764595

**The RMSE is 353.71.**

# Q7 {-}
Develop a MARS (Multivariate adaptive regression spline) model to predict *house_price* based on all the predictors except *house_id*. Allow a maximum of two degree interaction. Use the datasets with the *'_train'* suffix to develop the model. Do not transform any predictors. What is the RMSE of the model on test data?

*(4 points for code, 1 point for answer)*

In [64]:
X = train.drop(columns =[ 'house_price', 'house_id'])
y = train['house_price']

earth_model = Earth(max_terms = 500, max_degree = 2)
earth_model.fit(X, y)
np.sqrt(mean_squared_error(earth_model.predict(test.drop(columns = ['house_price', 'house_id'])), test.house_price))

C:\Users\nvsim\anaconda3\lib\site-packages\pyearth\earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
C:\Users\nvsim\anaconda3\lib\site-packages\pyearth\earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


388.9160524789007

**The RMSE is 388.916.**

# Q8 {-} 
Update the MARS model in Q7 to obtain a RMSE of less than 325 units on test data. \
**Hint:** Log transform *house_price*, and optimize the degree of the model

*(2 points for code, 1 point for answer)*

In [71]:
X = train.drop(columns = ['house_price','house_id'])
y = np.log(train['house_price'])

earth_model = Earth(max_terms = 500, max_degree = 1)
earth_model.fit(X, y)
np.sqrt(mean_squared_error(np.exp(earth_model.predict(test.drop(columns = ['house_price','house_id']))), test.house_price))

C:\Users\nvsim\anaconda3\lib\site-packages\pyearth\earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
C:\Users\nvsim\anaconda3\lib\site-packages\pyearth\earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


332.69332017786934

The RMSE for the MARS mode  is 332.

# Q9 {-}
Which predictors are rejected by the model developed in Q8?

*(1 points for code, 1 point for answer)*

In [72]:
print(earth_model.summary())

Earth Model
-------------------------------------------------
Basis Function             Pruned  Coefficient   
-------------------------------------------------
(Intercept)                No      6.33144       
h(distance_MRT-1144.44)    No      -0.000177672  
h(1144.44-distance_MRT)    No      0.00078818    
latitude                   Yes     None          
h(house_age-27.6)          No      0.0307957     
h(27.6-house_age)          No      0.0254309     
number_convenience_stores  No      0.0299533     
h(latitude-24.9702)        No      24.1518       
h(24.9702-latitude)        No      -15.4567      
h(latitude-24.9784)        Yes     None          
h(24.9784-latitude)        Yes     None          
h(latitude-24.9556)        Yes     None          
h(24.9556-latitude)        Yes     None          
-------------------------------------------------
MSE: 0.1040, GCV: 0.1196, RSQ: 0.8285, GRSQ: 0.8044


**It pruned latitude and longitude.**

# Q10 {-}
The data for this question is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls, where bank clients were called to subscribe for a term deposit. 

There is one train data - *train.csv*, which you will use to develop a model. There are two test datasets - *test1.csv* and *test2.csv*, which you will use to test your model. Each dataset has the following attributes about the clients called in the marketing campaign:

1) *age*: Age of the client\
2) *education*: Education level of the client \
3) *day*: Day of the month the call is made\
4) *month*: Month of the call \
5) *y*: did the client subscribe to a term deposit? \
6) *duration*: Call duration, in seconds. This attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call *y* is obviously known. Thus, this input should only be included for inference purposes and should be discarded if the intention is to have a realistic predictive model.

(Raw data source: [Source](https://archive.ics.uci.edu/ml/datasets/bank+marketing). Do not use the raw data source for this assingment. It is just for reference.)

Develop a **generalized additive model (GAM)** to predict the probability of a client subscribing to a term deposit based on *age, education, day* and *month*. The model must have: \
(a)  **Minimum overall classification accuracy of 75%** among the classifcation accuracies on *train.csv*, *test1.csv* and *test2.csv*. \
(b) **Maximum false negative rate of 45%** among the false negative rates on *train.csv*, *test1.csv* and *test2.csv*. 

Print the confusion matrices for all the three datasets - *train.csv*, *test1.csv* and *test2.csv*, along with the overall classification accuracies, and false negative rates.


Note that: \
(i) You cannot use *duration* as a predictor. The predictor is not useful for prediction because its value is determined after the marketing call ends. However, after the call ends, we already know whether the client responded positively or negatively. That is why we have used *duration* only for inference in the previous questions. It helped us understand the effect of the length of the call on marketing success. \
(ii) One way to develop the model satisfying constrains (a) and (b) is to use **spline tranformations for *age* and *day*, and interacting *month* with all the predictors (including the spline transformations)**\
(iii) You are free to choose any value of thershold probability for classifying observations. However, you must use the same threshold on all the three datasets.

*(10 points for code)*